**Name: Vishakha Prakash Ratnakar**


# Task 1: 

Write a Python script using RDFlib (or similar) to transform the GeoNames data into RDF.

In [ ]:
!pip3 install rdflib
!pip3 install rdflib-jsonld
!pip3 install sparqlwrapper


In [ ]:
#loading requried packages and libraries

from rdflib import Graph, plugin
from SPARQLWrapper import SPARQLWrapper
from rdflib.serializer import Serializer
import pandas as pd #for handling csv and csv contents
from rdflib import Graph, Literal, RDF, URIRef, Namespace #basic RDF handling
from rdflib.namespace import FOAF , XSD #most common namespaces
import urllib.parse #for parsing strings to URI's

In [ ]:
#read file from google drive

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd

df= pd.read_csv("/content/gdrive/MyDrive/IE.txt",sep='\t', header=None)  

df.columns = ["geonameid","name","asciiname","alternatename","latitude","longitude","feature_class","feature_code",
              "country_code","cc2","admin1","admin2","admin3","admin4","population","elevation","dem","timezone","modification"]

df.head()

,geonameid,name,asciiname,alternatename,latitude,longitude,feature_class,feature_code,country_code,cc2,admin1,admin2,admin3,admin4,population,elevation,dem,timezone,modification
0,2635367,Tullyrossmearan,Tullyrossmearan,"Tullyrosmearn,Tullyrossmearan",54.33333,-7.98333,P,PPL,IE,GB,C,14.0,NaN,NaN,0,NaN,53,Europe/Dublin,2015-05-15
1,2642090,Mourne Beg River,Mourne Beg River,"Mourne Beg,Mourne Beg River",54.68333,-7.65000,H,STM,IE,"IE,GB",00,NaN,NaN,NaN,0,NaN,73,Europe/Dublin,2017-07-26
2,2643245,Lough Macnean Upper,Lough Macnean Upper,"Lough Macnean,Lough Macnean Upper",54.30000,-7.93333,H,LK,IE,"IE,GB",00,NaN,NaN,NaN,0,NaN,48,Europe/Dublin,2017-07-26
3,2649110,River Foyle,River Foyle,"An Feabhal,Foyle,River Foyle",54.98532,-7.33277,H,STM,IE,"IE,GB",NaN,NaN,NaN,NaN,0,NaN,0,Europe/Dublin,2015-08-09
4,2649929,River Erne,River Erne,River Erne,54.50534,-8.24507,H,STM,IE,NaN,NaN,NaN,NaN,NaN,0,NaN,-9999,Europe/Dublin,2011-10-19


In [ ]:
# Defining graph g and namespace for classed URI
g = Graph()

#subject URI
ID_URI = Namespace('http://example.org/geonameid/')

#Names predicate URI
name_URI = Namespace('http://example.org/Names/')

#co_ordinate predicate URI
co_ordinates_URI = Namespace('http://example.org/co_ordinates/')

#admin_code predicate URI
admin_code_URI = Namespace('http://example.org/admin_code/')

predicates = ["has_names","has_asciiName","has_alternateName",

              "has_latitude","has_longitude","has_featureClass","has_featureCode",

              "countryCode","has_cc2","has_admin1",

              "has_admin2","has_admin3","has_admin4",

              "hasPopulation","hasElevation","has_dem",

              "has_timeZone","has_modificationDate"]

baseURI = Namespace("http://example.org/")

In [ ]:
#defining subject, predicate, object triple and adding to graph g

for index, row in df.iterrows():
    
    subject = (ID_URI+str(row['geonameid']))
    
    g.add((URIRef(subject),URIRef(name_URI + predicates[0]),Literal(row['name'],datatype = XSD.string)))
    g.add((URIRef(subject),URIRef(name_URI+predicates[1]),Literal(row['asciiname'],datatype = XSD.string)))
    g.add((URIRef(subject),URIRef(name_URI+predicates[2]),Literal(row['alternatename'],datatype = XSD.string)))
    g.add((URIRef(subject),URIRef(co_ordinates_URI + predicates[3]),Literal(row['latitude'],datatype = XSD.float)))
    g.add((URIRef(subject),URIRef(co_ordinates_URI + predicates[4]),Literal(row['longitude'],datatype = XSD.float)))
    g.add((URIRef(subject),URIRef(baseURI+predicates[5]),Literal(row['feature_class'],datatype = XSD.string)))
    g.add((URIRef(subject),URIRef(baseURI+predicates[6]),Literal(row['feature_code'],datatype = XSD.string)))
    g.add((URIRef(subject),URIRef(baseURI+predicates[7]),URIRef(baseURI +'/countrycode/' + row['country_code'])))
    g.add((URIRef(subject),URIRef(admin_code_URI +predicates[8]),Literal(row['cc2'],datatype = XSD.string)))
    g.add((URIRef(subject),URIRef(admin_code_URI+predicates[9]),Literal(row['admin1'],datatype = XSD.string)))
    g.add((URIRef(subject),URIRef(admin_code_URI+predicates[10]),Literal(row['admin2'],datatype = XSD.string)))
    g.add((URIRef(subject),URIRef(admin_code_URI+predicates[11]),Literal(row['admin3'],datatype = XSD.string)))
    g.add((URIRef(subject),URIRef(admin_code_URI+predicates[12]),Literal(row['admin4'],datatype = XSD.string)))
    g.add((URIRef(subject),URIRef(baseURI+predicates[13]),Literal(row['population'],datatype = XSD.integer)))
    g.add((URIRef(subject),URIRef(baseURI+predicates[14]),Literal(row['elevation'],datatype = XSD.float)))
    g.add((URIRef(subject),URIRef(baseURI+predicates[15]),Literal(row['dem'],datatype = XSD.integer)))
    g.add((URIRef(subject),URIRef(baseURI+predicates[16]),Literal(row['timezone'],datatype = XSD.string)))
    g.add((URIRef(subject),URIRef(baseURI+predicates[17]),Literal(row['modification'],datatype = XSD.date)))

In [ ]:
# serializing the graph into turtle format
g.serialize("geonames_rdf.ttl",format='turtle')



<Graph identifier=Na7feef26567d45cd96d19bcec8494c51 (<class 'rdflib.graph.Graph'>)>

In [ ]:
#Some triples

cnt = 0
for subject, predicate, obj in g:
    if cnt != 20:

          print(str(subject)+" ---> "+str(predicate)+" ---> "+str(obj))
          cnt += 1
    else:
          break;

http://example.org/geonameid/2963665 ---> http://example.org/admin_code/has_admin3 ---> nan
http://example.org/geonameid/3308858 ---> http://example.org/Names/has_names ---> Sandfield
http://example.org/geonameid/12112600 ---> http://example.org/admin_code/has_admin2 ---> 31.0
http://example.org/geonameid/3307002 ---> http://example.org/has_featureClass ---> S
http://example.org/geonameid/3304914 ---> http://example.org/hasElevation ---> nan
http://example.org/geonameid/12105480 ---> http://example.org/co_ordinates/has_longitude ---> -8.7711
http://example.org/geonameid/7869966 ---> http://example.org/co_ordinates/has_latitude ---> 51.86292
http://example.org/geonameid/11821081 ---> http://example.org/admin_code/has_admin1 ---> M
http://example.org/geonameid/3312962 ---> http://example.org/has_timeZone ---> Europe/Dublin
http://example.org/geonameid/2966743 ---> http://example.org/has_featureCode ---> PPL
http://example.org/geonameid/10130416 ---> http://example.org/admin_code/has_admi

# Task 2
Download the Wikidata data snippet from Blackboard. You should develop an algorithm to find
entities in the two datasets that describe the same real-world location. You should report on
your algorithm and implement it in Python.



In [ ]:
# loading wikidata into graph

g_wikidata= Graph()
g_wikidata.parse("/content/gdrive/MyDrive/wikidata_ie.nt", format="nt")

<Graph identifier=Nd69c88b47a6f49fb9dab970231ffd0fe (<class 'rdflib.graph.Graph'>)>

In [ ]:
#load geoname data graph

geo_graph = Graph()
geo_graph.parse("geonames_rdf.ttl", format="ttl")

<Graph identifier=Nad8e4200404445a2b03b6b14d5bb9777 (<class 'rdflib.graph.Graph'>)>

In [ ]:
#extract names and latitude longitudes from GeoNames dataset
# convert the co-ordinates into float store it in tuple
#Store names + co-ordinates as dictionary where keys = names and values = co_ordinates

geoname=()
geo_cord = {}

query = """
SELECT  ?latitude ?longitude ?name
WHERE {
?geonameid <http://example.org/co_ordinates/has_latitude> ?latitude .
?geonameid <http://example.org/co_ordinates/has_longitude> ?longitude .
?geonameid <http://example.org/Names/has_names> ?name .
}"""


qres = geo_graph.query(query)
for row in qres:
    longitude= float(f"{row.longitude}")
    latitude= float(f"{row.latitude}")
    geoname=(longitude,latitude)
    geo_cord[f"{row.name}"]=geoname
   


In [ ]:
geo_cord

{'Windermere House': (-9.28842, 53.84709),
 'Towers Hotel': (-9.75586, 52.09976),
 'The Lake Of Shadows Hotel': (-7.46007, 55.13273),
 'Gresham Metropole Hotel And Leisure Centre Cork': (-8.46896, 51.90406),
 'Fitzpatrick Bunratty Shamrock': (-8.81965, 52.69851),
 'Baile Ricead': (-6.69503, 52.88083),
 'Quality Hotel & Leisure Centre': (-8.89617, 51.61733),
 'Faul': (-10.04004, 53.47755),
 'Ardbear': (-10.02506, 53.47924),
 'Ardbear Cottages': (-10.02528, 53.48391),
 'Faul Lough': (-10.03846, 53.47577),
 'Derrigmlagh': (-10.04112, 53.44456),
 'Derrygimla': (-10.06107, 53.44467),
 'Marconi Station World Heritage Site': (-10.02506, 53.44607),
 'Clifden Village': (-10.02738, 53.49182),
 'Cloghaunard': (-10.04236, 53.48848),
 "Hyland's Burren": (-9.15129, 53.11788),
 'Rosslare Europort Ferry': (-6.33482, 52.25322),
 'Irish Ferries': (-6.33481, 52.2524),
 'Stena Line': (-6.33432, 52.25262),
 'Rosslare Europort Train Station': (-6.33781, 52.25132),
 'Ballygillane Big': (-6.34435, 52.24572),


In [ ]:
#extract cordinates from wikidata and store as list

wikidata=[]

query = """
SELECT ?co_ordinates
WHERE {
?Qcode <http://www.wikidata.org/prop/direct/P625> ?co_ordinates .
}"""



qres = g_wikidata.query(query)
for row in qres:
    data=(f"{row.co_ordinates}")

    # to avoid any null values present using if condition. If the string starts with 'Point' only then
    #co_ordinates will split into latitude and longitude.
    if 'Point' in data:
        data=data.replace('Point(', '')
        data=data.replace(')', '')
        split_data=data.split()
        longitude=float(split_data[0]) #convert string to float
        latitude=float(split_data[1])   # convert string to float
        wiki_tuple=(longitude,latitude) 
        wikidata.append(wiki_tuple)   #store in tuple



In [ ]:
wikidata


[(-6.1958, 53.2888),
 (-6.25972, 53.3478),
 (-6.8811111111111, 52.8375),
 (-7.0, 54.4),
 (-6.2597222222222, 53.347777777778),
 (-8.5833333333333, 51.966666666667),
 (-9.05, 53.78),
 (-6.8333333333333, 52.666666666667),
 (-7.5, 53.5),
 (-7.5, 53.5),
 (-6.5, 53.833333333333),
 (-7.5, 53.25),
 (-8.4833, 54.2667),
 (-8.4833, 54.2667),
 (-7.919, 54.2347),
 (-8.7545, 52.6767),
 (-6.0, 52.0),
 (-6.6669, 53.2158),
 (-6.6669, 53.2158),
 (-7.7116, 52.3539),
 (-7.7116, 52.3539),
 (-7.43333333, 55.01666667),
 (-7.45, 54.1),
 (-8.51916667, 54.25086111),
 (-7.7, 53.1),
 (-6.569235, 53.141252),
 (-7.26694444, 52.53833333),
 (-9.9217305555556, 53.932761111111),
 (-6.498678, 53.688358),
 (-9.3025, 52.381388888889),
 (-7.65, 53.583333),
 (-7.5, 53.2667),
 (-7.85344444, 53.62675),
 (-6.26707, 53.350449),
 (-6.26707, 53.350449),
 (-6.8301, 53.4142),
 (-9.792, 51.686),
 (-7.9175782, 53.0967722),
 (-7.1463, 52.1588),
 (-6.463, 52.966),
 (-8.9772, 52.6001),
 (-8.10611111, 53.93805556),
 (-8.1747, 51.9088),
 

In [ ]:
real_location=[]

# comparing values of dictionary with list of co-ordinates
for key ,value in geo_cord.items():
    for wikipoints in wikidata:
        if wikipoints==value:
            real_location.append(key)

In [ ]:
len(real_location)

578

# Task 3

On either your translated GeoNames dataset or the Wikidata article, write SPARQL queries or
equivalent Python functions that can answer the following questions:

● What is the highest point in Ireland?

● Which city is called “Cloch na Rón” in Irish?

● How many places in Ireland are called “Ballymore”?


**What is the highest point in Ireland?**

Ans- 

As we have to find the highest point in Ireland,we'll filter out all the places that belong to the "Republic of Ireland".After that, by using property P2044 (elevation_above_sea_level), the elevation of each place is selected.Furthermore, the result is ordered in descending order and the LIMIT clause is used to select the top value.

In [ ]:
#What is the highest point in Ireland?
query = """

SELECT ?highest_point_name
WHERE {
?Qcode <http://www.wikidata.org/prop/direct/P1705> ?highest_point_name.
?Qcode <http://www.wikidata.org/prop/direct/P2044> ?elevation_above_sea_level.
?Qcode <http://www.wikidata.org/prop/direct/P17> ?country
FILTER(?country= <http://www.wikidata.org/entity/Q27>)

}
ORDER BY DESC(?elevation_above_sea_level)
LIMIT 1"""

qres = g_wikidata.query(query)
for row in qres:
  print(f"Highest point: {row.highest_point_name} ")

Highest point: British Isles 


**Which city is called “Cloch na Rón” in Irish?**

Ans- 

The FILTER regex() helps to find the pattern in the data. It works as a LIKE clause in MySQL. By using  object of <http://www.w3.org/2000/01/rdf-schema#label> that is name of the citiy in irish langugae, we can find out which city has irish name as cloch na ron. 

In [ ]:
#Which city is called “Cloch na Rón” in Irish

query1 = """
SELECT ?city
WHERE {

?Qcode <http://www.w3.org/2000/01/rdf-schema#label> ?irish_name.
FILTER regex(?irish_name , "^Cloch na Rón")
?Qcode <http://www.wikidata.org/prop/direct/P373> ?city

}
"""
qres = g_wikidata.query(query1)
for row in qres:
  print("Which city is called 'Cloch na Ron' in Irish:",row.city)

Which city is called 'Cloch na Ron' in Irish: Roundstone


**How many places in Ireland are called “Ballymore”?**

Ans- 

To begin, the data was used to select the country "republic of Ireland." Following that, the name "Ballymore" was entered into the filter as a name. The COUNT() function is useful for determining the precise number of rows in a result.

In [ ]:
#How many places in Ireland are called “Ballymore”?

query = """
SELECT (COUNT(?irish_name) AS ?cnt)
WHERE {

?Qcode <http://www.w3.org/2000/01/rdf-schema#label> ?irish_name.
?Qcode <http://www.wikidata.org/prop/direct/P17> ?country
FILTER(?country= <http://www.wikidata.org/entity/Q27> && ?irish_name= "Ballymore"@en)

}
"""
qres = g_wikidata.query(query)
for row in qres:
    print("number of places in Ireland that are called as Ballymore: ",row.cnt)

number of places in Ireland that are called as Ballymore:  21
